# Utils

In [1]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
from tqdm import tqdm
from itertools import product
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, r2_score
from sklearn.metrics import precision_recall_curve, roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [3]:
# Paths
path_labels = '/kaggle/input/fcmaps-metadata/labels.csv'
path_fcmaps = '/kaggle/input/fcmaps-processed'
path_fcmaps_augmented = '/kaggle/input/fcmaps-augmented-processed/FCmaps_augmented_processed'

# Load labels
df_labels = pd.read_csv(path_labels)

# Temporary:
to_exclude = ['3_S_5003', '4_S_5003', '4_S_5005', '4_S_5007', '4_S_5008']
df_labels = df_labels[~df_labels['ID'].isin(to_exclude)].reset_index(drop=True)

In [4]:
# Parameters
batch_size = 4
epochs = 10
lr = 1e-3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
task = 'classification'               # or 'regression'
n_folds = 10

if task == 'classification':
    criterion = nn.CrossEntropyLoss()
    n_classes = 2
    label_column = 'Group'
else:
    criterion = nn.MSELoss()
    n_classes = 1
    #label_column = 'Age'

# Dataset

In [5]:
class FCDataset(Dataset):
    def __init__(self, data_dir, df_labels, label_column, task, transform=None):
        self.data_dir = data_dir
        self.df_labels = df_labels.reset_index(drop=True)
        self.label_column = label_column
        self.task = task
        self.transform = transform

        # Dictionary for mapping strings to indices if labels are not numbers
        if not pd.api.types.is_numeric_dtype(self.df_labels[self.label_column]):
            unique_labels = sorted(self.df_labels[self.label_column].unique())
            self.label_mapping = {label: i for i, label in enumerate(unique_labels)}
        else:
            self.label_mapping = None

        self.samples = []

        # Loop over each row of the dataframe
        for _, row in self.df_labels.iterrows():
            subj_id = row['ID']

            if self.task == 'classification':
                # Classification: map the labels
                label = self.label_mapping[row[self.label_column]]

            else:
                # Regression: convert to float
                label = float(row[self.label_column])

            # Reconstruct the file paths
            file_path = os.path.join(data_dir, f"{subj_id}.processed.npy")
            if os.path.exists(file_path):
                self.samples.append((file_path, label))
            else:
                print(f"Missing file: {file_path}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_path, label = self.samples[idx]

        # Load and reshape the volume: (1, 91, 109, 91)
        volume = np.load(file_path)
        volume = np.expand_dims(volume, axis=0)

        # Covert volume into a tensor
        x = torch.tensor(volume, dtype=torch.float32)

        # Convert the label into a tensor
        if self.task == 'classification':
            y = torch.tensor(label, dtype=torch.long)
        else:
            y = torch.tensor(label, dtype=torch.float32)

        if self.transform:
            x = self.transform(x)

        return x, y

In [6]:
class AugmentedFCDataset(Dataset):
    def __init__(self, data_dir, df_labels, label_column, task, transform=None):

        self.data_dir = data_dir
        self.df_labels = df_labels.reset_index(drop=True)
        self.label_column = label_column
        self.task = task
        self.transform = transform

        # Mapping
        if not pd.api.types.is_numeric_dtype(self.df_labels[self.label_column]):
            unique_labels = sorted(self.df_labels[self.label_column].unique())
            self.label_mapping = {label: i for i, label in enumerate(unique_labels)}
        else:
            self.label_mapping = None

        self.samples = []

        for _, row in self.df_labels.iterrows():
            subj_id = row['ID']

            if self.task == 'classification':
                label = self.label_mapping[row[self.label_column]]
            else:
                label = float(row[self.label_column])

            # Loop over each subject folder and list each augmentation
            subject_folder = os.path.join(data_dir, subj_id)
            if os.path.isdir(subject_folder):
                for file in os.listdir(subject_folder):
                    if file.endswith('.npy'):
                        file_path = os.path.join(subject_folder, file)
                        self.samples.append((file_path, label))
            else:
                print(f"Warning: missing augmented folder for subject {subj_id}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        file_path, label = self.samples[idx]

        # Load and reshape the volume: (1, 91, 109, 91)
        volume = np.load(file_path)
        volume = np.expand_dims(volume, axis=0)

        x = torch.tensor(volume, dtype=torch.float32)

        if self.task == 'classification':
            y = torch.tensor(label, dtype=torch.long)
        else:
            y = torch.tensor(label, dtype=torch.float32)

        if self.transform:
            x = self.transform(x)

        return x, y

# CNN Models

## Network

In [7]:
from torchvision.models.video import r3d_18

class SimpleCNN3D(nn.Module):
    def __init__(self, n_classes):
        super(SimpleCNN3D, self).__init__()
        self.conv1 = nn.Conv3d(1, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm3d(16)
        self.pool1 = nn.MaxPool3d(2)

        self.conv2 = nn.Conv3d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm3d(32)
        self.pool2 = nn.MaxPool3d(2)

        self.conv3 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm3d(64)
        self.pool3 = nn.AdaptiveAvgPool3d(1)

        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(64, n_classes)

    def forward(self, x):
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        return self.fc(x)


class ResNet3D(nn.Module):
    def __init__(self, n_classes):
        super(ResNet3D, self).__init__()
        self.model = r3d_18(pretrained=False)
        self.model.stem[0] = nn.Conv3d(1, 64, kernel_size=(3,7,7), stride=(1,2,2), padding=(1,3,3), bias=False)
        self.model.fc = nn.Linear(self.model.fc.in_features, n_classes)

    def forward(self, x):
        return self.model(x)

## Training, Validation and Testing loops

In [8]:
def train(train_loader, model, criterion, optimizer, device):
    # Enable training mode (dropout, batchnorm updates)
    model.train()
    running_loss = 0.0

    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        # Reset gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(x_batch)

        # Compute loss
        loss = criterion(outputs, y_batch)

        # Backpropagation
        loss.backward()

        # Update weights
        optimizer.step()

        running_loss += loss.item() * x_batch.size(0)

    train_loss = running_loss / len(train_loader.dataset)
    return train_loss

In [9]:
def validate(val_loader, model, criterion, device, task):
    model.eval()
    running_loss = 0.0
    correct = 0

    with torch.no_grad():
        for x_val, y_val in val_loader:
            x_val, y_val = x_val.to(device), y_val.to(device)
            outputs = model(x_val)
            loss = criterion(outputs, y_val)
            running_loss += loss.item() * x_val.size(0)

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y_val).sum().item()

    val_loss = running_loss / len(val_loader.dataset)
    val_accuracy = correct / len(val_loader.dataset)
    return val_loss, val_accuracy

In [10]:
def evaluate(model, loader, task, device):
    model.eval()
    true_labels, pred_labels = [], []

    # Disable gradient computation
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)

            # Convert output to prediction
            if task == 'classification':
                preds = torch.argmax(outputs, dim=1)
            else:
                preds = outputs.squeeze()

            # Store true and predicted values
            true_labels.extend(y.cpu().numpy())
            pred_labels.extend(preds.cpu().numpy())

    return np.array(true_labels), np.array(pred_labels)

# Data

Load some files for example

In [11]:
img3D = np.load('/kaggle/input/fcmaps-processed/002_S_4654.processed.npy')

print(img3D.shape)
print(img3D.dtype)

(91, 109, 91)
float32


## Pairwise classification

- ADNI + CBS = 116
- CBS + PSP = 105
- ADNI + PSP = 133

In [12]:
group1 = 'ADNI'
group2 = 'PSP'

df_pair = df_labels[df_labels['Group'].isin([group1, group2])].reset_index(drop=True)

In [13]:
df_pair.shape

(129, 5)

## Splitting

In [14]:
train_df, test_df = train_test_split(
    df_pair,
    test_size=0.2,
    stratify=df_pair['Group'],
    random_state=42
)

Check the balance between groups

In [15]:
print(train_df['Group'].value_counts())
print(test_df['Group'].value_counts())

Group
ADNI    57
PSP     46
Name: count, dtype: int64
Group
ADNI    15
PSP     11
Name: count, dtype: int64


Example of augmentation

In [16]:
train_dataset = AugmentedFCDataset(path_fcmaps_augmented, train_df, label_column, task)

In [17]:
print("Shape before augmentation: ", train_df.shape )
print("Shape after augmentation: ", len(train_dataset) )

Shape before augmentation:  (103, 5)
Shape after augmentation:  1030


# Training

In [18]:
# -------------------- PARAMETERS --------------------

# Search spaces for each hyperparameter
batch_sizes = [4, 8, 16]   
lrs = [1e-4, 5e-4, 1e-3]  
epochs_list = [10]  
weight_decays = [0, 1e-5]  
optimizers = ['adam', 'sgd']  

# Random combinations
n_trials = 20

results = []
all_train_val_losses = []

# -------------------- SPLIT TRAIN/VAL --------------------

subjects = train_df['ID'].values
labels = train_df[label_column].values

train_subj, val_subj = train_test_split(subjects, stratify=labels, test_size=0.2, random_state=42)

df_train = train_df[train_df['ID'].isin(train_subj)]
df_val = train_df[train_df['ID'].isin(val_subj)]

# -------------------- RANDOM SEARCH --------------------

for trial in range(n_trials):
    # Extract a random combination of hyperparameters
    batch_size = random.choice(batch_sizes)
    lr = random.choice(lrs)
    epochs = random.choice(epochs_list)
    weight_decay = random.choice(weight_decays)
    optimizer_type = random.choice(optimizers)

    print(f"\nTrial {trial+1}/{n_trials} -> batch_size={batch_size}, lr={lr}, "
          f"epochs={epochs}, weight_decay={weight_decay}, optimizer={optimizer_type}")

    # Dataset e DataLoader
    train_dataset = AugmentedFCDataset(path_fcmaps_augmented, df_train, label_column, task)
    val_dataset = FCDataset(path_fcmaps, df_val, label_column, task)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Model
    #model = SimpleCNN3D(n_classes=n_classes).to(device)
    model = ResNet3D(n_classes=n_classes).to(device)
        
    # Optimizer
    if optimizer_type == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_type == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
    else:
        raise ValueError("Unsupported optimizer")

    # Scheduler 
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    # -------------------- TRAINING --------------------

    best_accuracy = -float('inf')
    best_val_loss = float('inf')
    
    for epoch in range(epochs):
        train_loss = train(train_loader, model, criterion, optimizer, device)
        val_loss, val_accuracy = validate(val_loader, model, criterion, device, task)
    
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | "
              f"Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.4f}")
    
        scheduler.step(val_loss)
    
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_val_loss = val_loss
            torch.save(model.state_dict(), f"best_model_trial{trial+1}.pt")


    # Save best results
    results.append({
    'trial': trial + 1,
    'batch_size': batch_size,
    'lr': lr,
    'epochs': epochs,
    'weight_decay': weight_decay,
    'optimizer': optimizer_type,
    'best_val_accuracy': best_accuracy,
    'best_val_loss': best_val_loss
})


Trial 1/20 -> batch_size=16, lr=0.0001, epochs=10, weight_decay=1e-05, optimizer=adam


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/10 | Train Loss: 0.6491 | Val Loss: 0.6745 | Val Accuracy: 0.5714
Epoch 2/10 | Train Loss: 0.3939 | Val Loss: 2.9362 | Val Accuracy: 0.6667
Epoch 3/10 | Train Loss: 0.2266 | Val Loss: 2.5189 | Val Accuracy: 0.7619
Epoch 4/10 | Train Loss: 0.1496 | Val Loss: 1.4789 | Val Accuracy: 0.6190
Epoch 5/10 | Train Loss: 0.0470 | Val Loss: 2.5404 | Val Accuracy: 0.6667
Epoch 6/10 | Train Loss: 0.0106 | Val Loss: 2.3389 | Val Accuracy: 0.6190
Epoch 7/10 | Train Loss: 0.0072 | Val Loss: 2.3724 | Val Accuracy: 0.6190
Epoch 8/10 | Train Loss: 0.0062 | Val Loss: 2.3762 | Val Accuracy: 0.5714
Epoch 9/10 | Train Loss: 0.0049 | Val Loss: 2.4991 | Val Accuracy: 0.6667
Epoch 10/10 | Train Loss: 0.0035 | Val Loss: 2.3975 | Val Accuracy: 0.6190

Trial 2/20 -> batch_size=4, lr=0.0001, epochs=10, weight_decay=0, optimizer=sgd
Epoch 1/10 | Train Loss: 0.6927 | Val Loss: 0.6672 | Val Accuracy: 0.6190
Epoch 2/10 | Train Loss: 0.6658 | Val Loss: 0.7354 | Val Accuracy: 0.6190
Epoch 3/10 | Train Loss: 0.666

In [19]:
df_results = pd.DataFrame(results)

print("\n==== Ranking for Accuracy ====")
df_acc = df_results.sort_values(by='best_val_accuracy', ascending=False).reset_index(drop=True)
df_acc['rank_accuracy'] = df_acc.index + 1
print(df_acc[['trial', 'best_val_accuracy', 'batch_size', 'lr', 'epochs', 'weight_decay', 'optimizer']])

print("\n==== Ranking for Loss ====")
df_loss = df_results.sort_values(by='best_val_loss', ascending=True).reset_index(drop=True)
df_loss['rank_loss'] = df_loss.index + 1
print(df_loss[['trial', 'best_val_loss', 'batch_size', 'lr', 'epochs', 'weight_decay', 'optimizer']])

print("\n==== Ranking Accuracy-Loss ====")
df_results['score'] = (df_results['best_val_accuracy'] / df_results['best_val_loss'])
df_combined = df_results.sort_values(by='score', ascending=False).reset_index(drop=True)
df_combined['rank_combined'] = df_combined.index + 1
print(df_combined[['trial', 'best_val_accuracy', 'best_val_loss', 'score','batch_size', 'lr', 'epochs', 'weight_decay', 'optimizer']])


==== Ranking for Accuracy ====
    trial  best_val_accuracy  batch_size      lr  epochs  weight_decay  \
0       4           0.809524          16  0.0010      10       0.00000   
1      16           0.809524           8  0.0005      10       0.00001   
2       5           0.761905           4  0.0005      10       0.00000   
3       1           0.761905          16  0.0001      10       0.00001   
4      17           0.761905           4  0.0001      10       0.00001   
5      19           0.761905           8  0.0010      10       0.00000   
6      20           0.761905           4  0.0001      10       0.00001   
7      18           0.761905           4  0.0001      10       0.00000   
8      15           0.761905          16  0.0010      10       0.00000   
9       2           0.714286           4  0.0001      10       0.00000   
10      3           0.714286           4  0.0001      10       0.00000   
11      6           0.714286          16  0.0005      10       0.00001   
12    

# Evaluation

In [20]:
best_trial = df_results.iloc[0]['trial']

best_model_path = f"best_model_trial{int(best_trial)}.pt"
print(f"Best model path: {best_model_path}")

Best model path: best_model_trial1.pt


In [21]:
test_dataset = FCDataset(path_fcmaps,test_df,label_column,task)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [22]:
# Load the model
model = ResNet3D(n_classes=n_classes).to(device)
model.load_state_dict(torch.load(best_model_path, weights_only=True))
model.eval()

# Test
y_true, y_pred = evaluate(model, test_loader, task, device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [23]:
# Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
try:
    roc_auc = roc_auc_score(y_true, y_pred)
except ValueError:
    roc_auc = float('nan')  # AUC not computable if only 1 class present

print("\n--- Test Metrics ---")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-score : {f1:.4f}")
print(f"ROC AUC  : {roc_auc:.4f}")

NameError: name 'precision_score' is not defined

In [ ]:
# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title("Test Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()